# Introduction to Pinjected

Pinjected has multiple features that are individually useful to start with.
1. Dependency Injection Concepts
    - What is Dependency Injection?
    - Constructor Injection
    - `Design` object and object creation.
        - Design
        - Injected
        - Resolver
    - Decorators to create `Design` object
2. Advanced Topics
    - Proxied Variables
3. Entrypoint Functionalities
4. IDE Supports

# What is Dependency Injection?

# Example with CIFAR-10
We describe how a typical machine learning code can be written with pinjected to increase modularity of the code.
First we show an original pytorch tutorial script to train and test a model with CIFAR-10 dataset.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import imshow

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:  # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

dataiter = iter(testloader)
images, labels = next(dataiter)

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

net = Net()
net.load_state_dict(torch.load(PATH, weights_only=True))
outputs = net(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Here, a simple neural network is trained and tested with CIFAR-10 dataset.
This gets the job done, however, we will surely need some more flexibility in the future.
For example, we might want to change the neural network architecture or optimizer or loss function.

A typical approach for introducing a config object and pass it around everywhere, which we do not recommend.
Let's rewrite the code to use config object, as typically seen in many machine learning repositories.


# Typical ML code with config loading

```json
{
  "data": {
    "dataset": "CIFAR10",
    "batch_size": 4,
    "num_workers": 2,
    "root": "./data"
  },
  "model": {
    "name": "SimpleCNN",
    "params": {
      "conv1_out_channels": 6,
      "conv1_kernel_size": 5,
      "conv2_out_channels": 16,
      "conv2_kernel_size": 5,
      "fc1_out_features": 120,
      "fc2_out_features": 84,
      "num_classes": 10
    }
  },
  "training": {
    "epochs": 2,
    "loss": "CrossEntropyLoss",
    "optimizer": {
      "name": "SGD",
      "params": {
        "lr": 0.001,
        "momentum": 0.9
      }
    }
  },
  "paths": {
    "save_model": "./cifar_net.pth"
  }
}
```

In [ ]:
import json
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import imshow

# Load configuration.
with open('config.json', 'r') as f:
    config = json.load(f)

# Define transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Select dataset
dataset_name = config['data']['dataset']
if dataset_name == "CIFAR10":
    trainset = torchvision.datasets.CIFAR10(root=config['data']['root'], train=True,
                                            download=True, transform=transform)
    testset = torchvision.datasets.CIFAR10(root=config['data']['root'], train=False,
                                           download=True, transform=transform)
    num_classes = 10
    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
else:
    raise ValueError(f"Unsupported dataset: {dataset_name}")

trainloader = torch.utils.data.DataLoader(trainset, batch_size=config['data']['batch_size'],
                                          shuffle=True, num_workers=config['data']['num_workers'])
testloader = torch.utils.data.DataLoader(testset, batch_size=config['data']['batch_size'],
                                         shuffle=False, num_workers=config['data']['num_workers'])


# Define models
class SimpleCNN(nn.Module):
    def __init__(self, model_config):
        super().__init__()
        self.conv1 = nn.Conv2d(3, model_config['conv1_out_channels'], model_config['conv1_kernel_size'])
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(model_config['conv1_out_channels'], model_config['conv2_out_channels'],
                               model_config['conv2_kernel_size'])
        self.fc1 = nn.Linear(model_config['conv2_out_channels'] * 5 * 5, model_config['fc1_out_features'])
        self.fc2 = nn.Linear(model_config['fc1_out_features'], model_config['fc2_out_features'])
        self.fc3 = nn.Linear(model_config['fc2_out_features'], model_config['num_classes'])

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Select model
model_name = config['model']['name']
if model_name == "SimpleCNN":
    net = SimpleCNN(config['model']['params'])
else:
    raise ValueError(f"Unsupported model: {model_name}")

# Select loss function
loss_name = config['training']['loss']
if loss_name == "CrossEntropyLoss":
    criterion = nn.CrossEntropyLoss()
else:
    raise ValueError(f"Unsupported loss function: {loss_name}")

# Select optimizer
optimizer_config = config['training']['optimizer']
optimizer_name = optimizer_config['name']
if optimizer_name == "SGD":
    optimizer = optim.SGD(net.parameters(), **optimizer_config['params'])
elif optimizer_name == "Adam":
    optimizer = optim.Adam(net.parameters(), **optimizer_config['params'])
else:
    raise ValueError(f"Unsupported optimizer: {optimizer_name}")

# Training loop
for epoch in range(config['training']['epochs']):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

PATH = config['paths']['save_model']
torch.save(net.state_dict(), PATH)

# Test the network
dataiter = iter(testloader)
images, labels = next(dataiter)

imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

net = SimpleCNN(config['model']['params'])
net.load_state_dict(torch.load(PATH, weights_only=True))
outputs = net(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Now this is the code we typically see in ML community repositories.
The code is now configurable to use other models and datasets.

However, notice the code is now contaminated with configuration loading with many 'if' statements.
The code is now hard to grance over and understand the main logic due to the code mixed with configuration loading.

The bad thing is that it is now hard to modify the code for other purposes.
Suppose if you want to evaluate the model without training, we need to rewrite the initialization process to only initialize what is needed. In this case, training related stuff such as optimizer and loss function, as well as their associated configuration are not needed and we want to strip it off for performance and readability.

This is a tiresome work to do and introduces a lot of redundant code.





# Pinjected version

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import imshow
from pinjected import *


# Define transforms
@instance
def transform_default():
    return transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])


# Dataset providers
@injected
def get_cifar10(data_root, transform, /, train=True):
    return torchvision.datasets.CIFAR10(root=data_root, train=train,
                                        download=True, transform=transform)


@injected
def get_dataloader(dataset, batch_size, num_workers, /, shuffle=True):
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                       shuffle=shuffle, num_workers=num_workers)


# Model definition
class SimpleCNN(nn.Module):
    def __init__(self, conv1_out_channels, conv1_kernel_size, conv2_out_channels,
                 conv2_kernel_size, fc1_out_features, fc2_out_features, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(3, conv1_out_channels, conv1_kernel_size)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(conv1_out_channels, conv2_out_channels, conv2_kernel_size)
        self.fc1 = nn.Linear(conv2_out_channels * 5 * 5, fc1_out_features)
        self.fc2 = nn.Linear(fc1_out_features, fc2_out_features)
        self.fc3 = nn.Linear(fc2_out_features, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Loss function provider
@instance
def criterion__cross_entropy():
    return nn.CrossEntropyLoss()


@instance
def optimizer_sgd(model, learning_rate, momentum=0.9):
    return optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)


@instance
def optimizer_adam(model, learning_rate):
    return optim.Adam(model.parameters(), lr=learning_rate)


# Training function
@injected
def train(model, trainloader, criterion, optimizer, epochs, /):
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 2000 == 1999:
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0

    print('Finished Training')


# Testing function
@injected
def test(model, testloader, classes, /):
    dataiter = iter(testloader)
    images, labels = next(dataiter)

    imshow(torchvision.utils.make_grid(images))
    print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

    outputs = model(images)
    _, predicted = torch.max(outputs, 1)

    print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                                  for j in range(4)))

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')


# Main execution function
@injected
def experiment(train, test,/):
    train()
    test()

run_experiment = experiment()    

# Configuration
setup_1:Design = design(
    data_root='./data',
    batch_size=4,
    num_workers=2,
    model=injected(SimpleCNN)(
        conv1_out_channels=6,
        conv1_kernel_size=5,
        conv2_out_channels=16,
        conv2_kernel_size=5,
        fc1_out_features=120,
        fc2_out_features=84,
        num_classes=10
    ),
    learning_rate=0.001,
    epochs=2,
    classes=('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'),
    transform=transform_default,
    trainset=get_cifar10(train=True),
    testset=get_cifar10(train=False),
    trainloader=get_dataloader(injected('trainset'), shuffle=True),
    testloader=get_dataloader(injected('testset'), shuffle=False),
    criterion=criterion__cross_entropy,
    optimizer=optimizer_sgd
)

# Run the experiment
if __name__ == "__main__":
    setup_1.provide(experiment())

Now, the main difference is that we have `@instance` and `injected` everywhere. Also, the wiring of object dependencies are specified in `setup_1` object.
The actual logic runs when we call `provide` method on `setup_1` object. The `provide` method creates all the required objects and wire them together to run the given `experiment` object.

Notice that the code has no `if` statement at all to handle configuration loading? This is because the configuration is now handled by `Design` object. This makes us possible to swap any objects with other objects easily.
Also, no configuration object is passed around everywhere, and we can clearly see the explicit dependencies required for each object.

The design instantiates only the required objects upon calling `provide` method.
We can now modify this code to run only the evaluation part, without rewriting whole initialization process.
Here is the example:




In [ ]:
setup_1.provide(test())

Now the Design object `setup_1` will automatically determine what must be initialized how, and runs the requested `test()`

Pinjected allows you to break down the code into modular parts and run only what is needed by automatically wiring them.
If you want to take a look at a model architecture, you can do `setup_1.provide('model')`. No other stuff such as training data loading or database initialization will be done, as long as it is not required for instantiating the model!




In [ ]:
setup_1.provide('model')



`@injected` and `@instance` makes sure to state what is needed for an object to be created by name. upon instantiation of annotated object, the dependencies are looked up by name inside the Design object, and resolved recursively.

For `@injected`, the name of function parameters before `/` is treated as its dependencies.
For `@instance`, all the name of parameters are treated as its dependencies.

Now, how do we change anything registered with `setup_1`?

Here an example to change the optimizer to Adam from SGD:




In [ ]:
setup_2 = setup_1 + design(
    optimizer=optimizer_adam
)

setup_2.provide('optimizer')

Additionally, we have a CLI to run the code based on pinjected.

```bash
python -m pinjected run <module.name_of_target> <module.name_of_design>
# can be simplified to;
pinjected run <module.name_of_target> <module.name_of_design>
```
So, if you save the previous training code as `train.py`, you can run the training code with the following command:

```bash 
pinjected run train.run_experiment train.setup_1
# or if you want to use adam,
pinjected run train.run_experiment train.setup_2
# you can also override other stuff:
pinjected run train.run_experiment train.setup_1 --epochs=10 --batch_size=10
# you can even override with pinjected object, by surrounding its path with '{}'
pinjected run train.run_experiment train.setup_1 --optimizer={train.optimizer_sgd}
```


